In [1]:
import nltk
import string
import scipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string

In [2]:
# Read in the data
reviews = pd.read_csv('Reviews.csv')

In [3]:
# Clean the data
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
reviews_without_stopwords = [word for word in reviews['Text'].str.split() if word not in stop_words]

In [4]:
# Set everything to lower case and remove punctuation
reviews_without_stopwords_or_punctuation = []
# reviews_without_stopwords_or_punctuation_full_list = []
for st in reviews_without_stopwords:
	tmp_array = []
	for word in st:
		if word not in string.punctuation:
			tmp_array.append(word.lower())
			# reviews_without_stopwords_or_punctuation_full_list.append(word.lower())
	reviews_without_stopwords_or_punctuation.append(tmp_array)

In [10]:
print(len(reviews_without_stopwords_or_punctuation))
print(len(reviews_without_stopwords_or_punctuation[1]))
print(reviews['Score'])

568454
31
0         5
1         1
2         4
3         2
4         5
         ..
568449    5
568450    2
568451    5
568452    5
568453    5
Name: Score, Length: 568454, dtype: int64


In [9]:
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(data_clean_raw, data[4].astype("int"), test_size=0.2, random_state=0)

0         5
1         1
2         4
3         2
4         5
         ..
568449    5
568450    2
568451    5
568452    5
568453    5
Name: Score, Length: 568454, dtype: int64
